In [6]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time
#from google.cloud import storage
#from google.oauth2 import service_account
#from google.cloud import dns
from io import StringIO
import os
Today = datetime.date.today()
Today_str = Today.strftime("%Y-%m-%d")

In [7]:
datetime.datetime.now().year
Current_year = datetime.datetime.now().year
years = list(range(Current_year, Current_year -1, -1))
all_matches = []

In [15]:
import requests
from bs4 import BeautifulSoup

# Send a GET request to the URL
url = "https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all h1 tags
h1_tags = soup.find_all("h1")

# Iterate through the h1 tags to find the desired line
for h1_tag in h1_tags:
    if "Bundesliga Stats" in h1_tag.text:
        line = h1_tag.text.strip()
        break

# Print the result
print(line)


2023-2024 Bundesliga Stats


In [16]:
year = line.split("-")[1].split()[0]
year

'2024'

In [18]:
Today = datetime.date(2024, 6, 1)
Today_str = Today.strftime("%Y-%m-%d")

# Send a GET request to the URL
url = "https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find all h1 tags
h1_tags = soup.find_all("h1")

# Iterate through the h1 tags to find the desired line
for h1_tag in h1_tags:
    if "Bundesliga Stats" in h1_tag.text:
        line = h1_tag.text.strip()
        break

year = line.split("-")[1].split()[0]
all_matches = []

standings_url = "https://fbref.com/en/comps/20/2023-2024/2023-2024-Bundesliga-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text, features='lxml')
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

previous_season = soup.select("a.prev")[0].get("href")
standings_url = f"https://fbref.com{previous_season}"

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    data = requests.get(team_url)
    html_data = StringIO(data.text)
    Scores_Fixtures = pd.read_html(html_data, match="Scores & Fixtures")[0]
    Scores_Fixtures = Scores_Fixtures[['Date', 'Time', 'Comp', 'Round', 'Day', 'Venue', 'GF', 'GA', 'Opponent', 'Poss']]
    Scores_Fixtures = Scores_Fixtures[(Scores_Fixtures['Date'] <= Today_str) & (Scores_Fixtures['Date'] > (Today - datetime.timedelta(days=90)).strftime("%Y-%m-%d"))]

    soup = BeautifulSoup(data.text, features='lxml')
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    html_data = StringIO(data.text)
    shooting = pd.read_html(html_data, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()
    shooting = shooting[(shooting['Date'] <= Today_str) & (shooting['Date'] > (Today - datetime.timedelta(days=90)).strftime("%Y-%m-%d"))]

    soup = BeautifulSoup(data.text, features='lxml')
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if l and 'all_comps/keeper/' in l]
    data = requests.get(f"https://fbref.com{links[0]}")
    html_data = StringIO(data.text)
    goalkeeping = pd.read_html(html_data, match="Goalkeeping")[0]
    goalkeeping.columns = goalkeeping.columns.droplevel()
    goalkeeping = goalkeeping[(goalkeeping['Date'] <= Today_str) & (goalkeeping['Date'] > (Today - datetime.timedelta(days=90)).strftime("%Y-%m-%d"))]

    #try:
    team_data = Scores_Fixtures.merge(shooting[['Date', 'Sh']], on=['Date'])
    team_data = team_data.merge(goalkeeping[['Date', 'Save%']], on=['Date'])

    #except ValueError:
    #continue
    team_data = team_data[team_data["Comp"] == "Bundesliga"]

    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(20)

match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]

match_df[['gf', 'ga', 'poss', 'sh', 'save%']] = match_df[['gf', 'ga', 'poss', 'sh', 'save%']].astype(float)
match_df['season'] = match_df['season'].astype(int)

In [19]:
match_df

,date,time,comp,round,day,venue,gf,ga,opponent,poss,sh,save%,season,team
1,2024-03-10,19:30,Bundesliga,Matchweek 25,Sun,Home,2.0,0.0,Wolfsburg,76.0,19.0,100.0,2024,Bayer Leverkusen
3,2024-03-17,15:30,Bundesliga,Matchweek 26,Sun,Away,3.0,2.0,Freiburg,67.0,19.0,33.3,2024,Bayer Leverkusen
4,2024-03-30,15:30,Bundesliga,Matchweek 27,Sat,Home,2.0,1.0,Hoffenheim,67.0,34.0,50.0,2024,Bayer Leverkusen
6,2024-04-06,15:30,Bundesliga,Matchweek 28,Sat,Away,1.0,0.0,Union Berlin,65.0,19.0,NaN,2024,Bayer Leverkusen
8,2024-04-14,17:30,Bundesliga,Matchweek 29,Sun,Home,5.0,0.0,Werder Bremen,56.0,19.0,100.0,2024,Bayer Leverkusen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2024-04-20,15:30,Bundesliga,Matchweek 30,Sat,Away,2.0,0.0,Köln,42.0,12.0,100.0,2024,Darmstadt 98
6,2024-04-28,19:30,Bundesliga,Matchweek 31,Sun,Home,0.0,1.0,Heidenheim,45.0,11.0,80.0,2024,Darmstadt 98
7,2024-05-04,15:30,Bundesliga,Matchweek 32,Sat,Away,0.0,3.0,Wolfsburg,47.0,6.0,57.1,2024,Darmstadt 98
8,2024-05-12,15:30,Bundesliga,Matchweek 33,Sun,Home,0.0,6.0,Hoffenheim,41.0,10.0,14.3,2024,Darmstadt 98
